In [2]:
!pip install httpx --quiet



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:

import uuid
import json
import asyncio
import httpx

from sdk.types import Role
from common.message_utils import create_text_message   

# Router Agent RPC endpoint
ROUTER_RPC = "http://localhost:8010/rpc"



In [5]:
# ---------------------------------------------------------------------------
# Helper: Build a JSON-RPC message/send payload
# ---------------------------------------------------------------------------
def build_rpc_payload(prompt: str) -> dict:
    msg = create_text_message(prompt, role=Role.user)
    return {
        "jsonrpc": "2.0",
        "id": msg.messageId,
        "method": "message/send",
        "params": {"message": msg.model_dump()},
    }


# ---------------------------------------------------------------------------
# Helper: Extract final agent response from the Router Task object
# ---------------------------------------------------------------------------
def extract_final_answer(task_result: dict) -> str:
    history = task_result.get("history", [])
    agent_msgs = [m for m in history if m.get("role") == "agent"]

    if not agent_msgs:
        return "<no agent messages>"

    final_msg = agent_msgs[-1]
    parts = final_msg.get("parts") or []
    if not parts:
        return "<no text parts>"

    return parts[0].get("text") or "<empty>"


# ---------------------------------------------------------------------------
# One scenario execution
# ---------------------------------------------------------------------------
async def run_single_scenario(name: str, query: str):
    payload = build_rpc_payload(query)

    async with httpx.AsyncClient() as client:
        response = await client.post(ROUTER_RPC, json=payload, timeout=30.0)
        response.raise_for_status()
        result = response.json().get("result")

    print("=" * 100)
    print(f"SCENARIO : {name}")
    print(f"QUERY    : {query}")
    print("-" * 100)

    if result is None:
        print("No result returned from Router.")
    else:
        final_text = extract_final_answer(result)
        print("FINAL RESPONSE:")
        print(final_text)

    print("=" * 100)
    print()


# ---------------------------------------------------------------------------
# Full scenario suite
# ---------------------------------------------------------------------------
scenarios = [
    ("Simple: basic customer lookup", "Get customer information for ID 5"),
    ("Coordinated: data + support", "I'm customer 12345 and need help upgrading my account"),
    ("Complex: multi-constraint search", "Show me all active customers who have open tickets"),
    ("Escalation: urgent refund", "I've been charged twice, please refund immediately!"),
    ("Multi-intent: data update + history", "Update my email and show ticket history"),
]


async def run_all():
    for name, query in scenarios:
        await run_single_scenario(name, query)


await run_all()


SCENARIO : Simple: basic customer lookup
QUERY    : Get customer information for ID 5
----------------------------------------------------------------------------------------------------
FINAL RESPONSE:
Router summary:
Customer record: {'id': 1, 'name': 'Ana Customer', 'email': 'ana@example.com', 'status': 'active', 'created_at': '2025-12-02 23:43:39'}
Hi there — I reviewed the latest notes on your account.
I’ve read through the account history you mentioned.
Here’s what I recommend based on your request:
- Tell me any specific details you'd like us to verify or double-check.
- We can also set up a short follow-up if you need more help.
- If this is urgent, reply here and I’ll jump on it immediately.
If you'd like me to take action now, just reply to this message and I’ll handle it.

SCENARIO : Coordinated: data + support
QUERY    : I'm customer 12345 and need help upgrading my account
----------------------------------------------------------------------------------------------------
